# RMH Geospatial API - Quick Reference

**Date**: 09 DEC 2025  
**Author**: Robert and Geospatial Claude Legion

Condensed notebook covering the core Platform endpoints:
1. Health Check
2. Process Vector
3. Process Raster (single)
4. Process Large Raster (>1GB)
5. Process Raster Collection (multi-file)

## Setup

In [ ]:
import requests
import json
import time

# =============================================================================
# CONFIGURATION - All variables defined here
# =============================================================================

# Function App Base URL
BASE_URL = "https://rmhazuregeoapi-a3dma3ctfdgngwf6.eastus-01.azurewebsites.net"

# Storage Containers (Bronze = raw input, Silver = processed output)
BRONZE_RASTERS_CONTAINER = "bronze-rasters"
BRONZE_VECTORS_CONTAINER = "bronze-vectors"
SILVER_COGS_CONTAINER = "silver-cogs"

# STAC Collections
RASTER_COLLECTION_ID = "system-rasters"
VECTOR_COLLECTION_ID = "system-vectors"

# PostGIS Schema
POSTGIS_SCHEMA = "geo"

# =============================================================================
# Helper Functions
# =============================================================================

def api_call(method, endpoint, data=None, params=None, timeout=30):
    """Make API call and return formatted response."""
    url = f"{BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    
    print(f"\n{'='*60}")
    print(f"{method} {endpoint}")
    print(f"{'='*60}")
    
    if data:
        print(f"\nRequest Body:")
        print(json.dumps(data, indent=2))
    
    try:
        if method == "GET":
            response = requests.get(url, params=params, timeout=timeout)
        elif method == "POST":
            response = requests.post(url, json=data, headers=headers, timeout=timeout)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        print(f"\nStatus: {response.status_code}")
        
        try:
            result = response.json()
            print(f"\nResponse:")
            print(json.dumps(result, indent=2, default=str))
            return result
        except:
            print(f"\nResponse (text): {response.text[:500]}")
            return response.text
            
    except requests.exceptions.Timeout:
        print(f"\n❌ Request timed out (timeout={timeout}s)")
        return None
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

def check_job_status(job_id, max_polls=20, poll_interval=5):
    """Poll job status until completion or timeout."""
    print(f"\n{'='*60}")
    print(f"Polling job: {job_id}")
    print(f"{'='*60}")
    
    for i in range(max_polls):
        result = requests.get(f"{BASE_URL}/api/jobs/status/{job_id}", timeout=30).json()
        status = result.get("status", "unknown")
        stage = result.get("current_stage", "?")
        
        print(f"  [{i+1}/{max_polls}] Status: {status}, Stage: {stage}")
        
        if status in ["completed", "failed"]:
            print(f"\nFinal Result:")
            print(json.dumps(result, indent=2, default=str))
            return result
        
        time.sleep(poll_interval)
    
    print(f"\n⚠️ Polling timeout after {max_polls * poll_interval}s")
    return result

# Display configuration
print("=" * 60)
print("API CONFIGURATION")
print("=" * 60)
print(f"Base URL:              {BASE_URL}")
print(f"Bronze Rasters:        {BRONZE_RASTERS_CONTAINER}")
print(f"Bronze Vectors:        {BRONZE_VECTORS_CONTAINER}")
print(f"Silver COGs:           {SILVER_COGS_CONTAINER}")
print(f"Raster Collection:     {RASTER_COLLECTION_ID}")
print(f"Vector Collection:     {VECTOR_COLLECTION_ID}")
print(f"PostGIS Schema:        {POSTGIS_SCHEMA}")
print("=" * 60)

---
## 1. Health Check

Comprehensive system health check (~60s due to database, Service Bus, and storage checks).

In [ ]:
# Health Check (takes ~60s)
result = api_call("GET", "/api/health", timeout=90)

---
## 2. Process Vector

Submit a vector file (GeoJSON, Shapefile, GeoPackage) for ingestion into PostGIS.

In [ ]:
# Submit Vector
vector_request = {
    "dataset_id": "test-vectors",
    "resource_id": "geojson-8",
    "version_id": "v1",
    "container_name": BRONZE_VECTORS_CONTAINER,
    "file_name": "8.geojson",
    "service_name": "Test GeoJSON 8"
}

result = api_call("POST", "/api/platform/submit", vector_request)
vector_job_id = result.get("job_id") if result else None
print(f"\n📋 Job ID: {vector_job_id}")

In [ ]:
# Check Vector Job Status
if vector_job_id:
    check_job_status(vector_job_id)
else:
    print("⚠️ No job_id from previous cell")

---
## 3. Process Raster (Single File)

Submit a single raster file for COG conversion and STAC cataloging.

In [ ]:
# Submit Single Raster
raster_request = {
    "dataset_id": "dctest",
    "resource_id": "analysis",
    "version_id": "v2",
    "container_name": BRONZE_RASTERS_CONTAINER,
    "file_name": "dctest.tif",
    "service_name": "DC Test Analysis v2"
}

result = api_call("POST", "/api/platform/raster", raster_request)
raster_job_id = result.get("job_id") if result else None
print(f"\n📋 Job ID: {raster_job_id}")

In [ ]:
# Check Raster Job Status
if raster_job_id:
    check_job_status(raster_job_id)
else:
    print("⚠️ No job_id from previous cell")

---
## 4. Process Large Raster (>1GB)

Submit a large raster (>1GB). Platform automatically routes to `process_large_raster_v2`.

In [ ]:
# Submit Large Raster (auto-routes to process_large_raster_v2)
large_raster_request = {
    "dataset_id": "laos-dtm",
    "resource_id": "luang-prabang",
    "version_id": "2021",
    "container_name": BRONZE_RASTERS_CONTAINER,
    "file_name": "2021_Luang_Prabang_DTM.tif",
    "service_name": "Luang Prabang DTM 2021"
}

result = api_call("POST", "/api/platform/raster", large_raster_request)
large_raster_job_id = result.get("job_id") if result else None
print(f"\n📋 Job ID: {large_raster_job_id}")

In [ ]:
# Check Large Raster Job Status
if large_raster_job_id:
    check_job_status(large_raster_job_id, max_polls=30, poll_interval=10)  # Longer timeout for large files
else:
    print("⚠️ No job_id from previous cell")

---
## 5. Process Raster Collection (Multi-File)

Submit multiple raster files to be processed as a collection with MosaicJSON.

In [ ]:
# Submit Raster Collection
collection_request = {
    "dataset_id": "namangan-imagery",
    "resource_id": "aug2019",
    "version_id": "v1",
    "container_name": BRONZE_RASTERS_CONTAINER,
    "file_name": [  # Must be list with at least 2 files
        "namangan/namangan14aug2019_R1C1cog.tif",
        "namangan/namangan14aug2019_R1C2cog.tif",
        "namangan/namangan14aug2019_R2C1cog.tif",
        "namangan/namangan14aug2019_R2C2cog.tif"
    ],
    "service_name": "Namangan Imagery August 2019"
}

result = api_call("POST", "/api/platform/raster-collection", collection_request)
collection_job_id = result.get("job_id") if result else None
print(f"\n📋 Job ID: {collection_job_id}")

In [ ]:
# Check Raster Collection Job Status
if collection_job_id:
    check_job_status(collection_job_id, max_polls=30, poll_interval=10)  # Longer timeout for multi-file
else:
    print("⚠️ No job_id from previous cell")

---
## Quick Reference: Manual Job Status Check

Use this cell to check any job by ID.

In [ ]:
# Manual Job Status Check
# Replace with your job_id
manual_job_id = "YOUR_JOB_ID_HERE"

if manual_job_id != "YOUR_JOB_ID_HERE":
    check_job_status(manual_job_id)
else:
    print("⚠️ Replace 'YOUR_JOB_ID_HERE' with an actual job_id")